### Block particle filter

* Algorithm 2 of https://projecteuclid.org/download/pdfview_1/euclid.aoap/1438261054

In [1]:
# Calling libraries:
from __future__ import division
%matplotlib inline
import numpy as np, time, matplotlib.pyplot as plt, math, pandas, numpy.random as npr, multiprocessing as mp, timeit
from pylab import plot, show, legend
from time import time
from scipy.stats import *
from tqdm import trange
from ecology_functions import *

In [2]:
T = 100
I = 7  # number of locations
J = 3  # number of species
K = 5   # number of latent factors

In [3]:
lmbda = npr.randn(J,K)
alpha = npr.randn(J)
c = 0
phi = 0.5
logsigmasq = 0
x_0 = npr.randn(I,K)

theta = [alpha, lmbda, c, phi, logsigmasq]
Y, X = simulate_data(x_0, T, J, theta)

In [4]:
# n_particles = 1000
# %timeit block_pf(Y, x_0, n_particles, theta, calc_grad=True)
# %timeit bootstrap_PF_grad(x_0, n_particles, theta, Y, calc_grad=True)
# %timeit block_pf(Y, x_0, n_particles, theta, calc_grad=False)
# %timeit bootstrap_PF_grad(x_0, n_particles, theta, Y, calc_grad=False)

In [5]:
# n_particles = 5000
# %timeit block_pf(Y, x_0, n_particles, theta, calc_grad=True)
# %timeit bootstrap_PF_grad(x_0, n_particles, theta, Y, calc_grad=True)
# %timeit block_pf(Y, x_0, n_particles, theta, calc_grad=False)
# %timeit bootstrap_PF_grad(x_0, n_particles, theta, Y, calc_grad=False)

In [6]:
def f_block(n_particles) :
    return block_pf(Y, x_0, n_particles, theta, True)
def f_bootstrap(n_particles) :
    return bootstrap_PF_grad(x_0, n_particles, theta, Y, True)

n_particles = 1000
rep = 100

pool = mp.Pool(10)
results_block = pool.map(f_block, [n_particles for n_particles in [n_particles]*rep])
results_bootstrap = pool.map(f_bootstrap, [n_particles for n_particles in [n_particles]*rep])
pool.close()

alpha_grad, lmbda_grad, logNC = np.zeros((rep,2,J)), np.zeros((rep,2,J,K)), np.zeros((rep,2))
for r in range(rep) :
    logNC[r,0], logNC[r,1] = results_block[r][0], results_bootstrap[r][0]
    alpha_grad[r,0], lmbda_grad[r,0] = results_block[r][1][0:2]
    alpha_grad[r,1], lmbda_grad[r,1] = results_bootstrap[r][1][0:2]

In [7]:
print(np.mean(logNC, 0))
print(np.var(logNC, 0))

[-1359.42972957 -1364.89120557]
[18.75359131 42.88469646]


In [8]:
print(np.mean(lmbda_grad, 0))

[[[  59.48825403    7.71370937  164.58877937 -108.68559801   21.41896423]
  [ -10.84948687  -15.88966041 -278.13151292   -8.64911008   50.31447991]
  [  19.83078394   72.31279036   31.45786556 -138.46146446   97.89600083]]

 [[  52.0844303     9.03707694  151.34140527  -99.33274024   24.38834825]
  [ -17.40165355  -10.78164744 -260.57872435    0.87327301   45.90480651]
  [  15.07683306   62.62050332   22.08697811 -124.03373364   88.81739684]]]


In [9]:
print(np.var(lmbda_grad, 0))

[[[ 33.27912111  40.15813615  19.08170796  28.32807163  33.26862009]
  [ 45.75859341  55.51169955  25.65181174  39.86583045  53.44954772]
  [ 45.77168139  50.27838403  27.90769628  37.57061277  47.49222174]]

 [[633.5324938  624.65845893 202.40141309 311.98903828 559.61343726]
  [643.21026717 760.87841111 251.4164623  393.42823084 658.487638  ]
  [759.27975252 951.54225911 323.9643154  517.71511433 682.42526988]]]


### Something strange. This should have worked. Double-check!


In [6]:
n_particles = 1000
particles = npr.randn(n_particles,I,K) 
propagated_particles = npr.randn(*np.shape(particles))
y = Y[1]

In [7]:
%timeit alpha_grad_blockPF(y, theta, propagated_particles, particles)
%timeit lmbda_grad_blockPF(y, theta, propagated_particles, particles)
%timeit c_grad_blockPF(y, theta, propagated_particles, particles)
%timeit phi_grad_blockPF(y, theta, propagated_particles, particles)
%timeit logsigmasq_grad_blockPF(y, theta, propagated_particles, particles)

131 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.43 ms ± 7.05 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
240 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
303 µs ± 992 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
269 µs ± 688 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
